#### File for initial data exploration

In [13]:
import json
import pandas as pd

In [14]:
path = "../data/"
file = "stunda-terms.jsonl"

In [15]:
def read_jsonl_file(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            # Load each line as a JSON object
            json_object = json.loads(line)
            data.append(json_object)
    return data

#### Read data and show an entry

In [16]:
data = read_jsonl_file(path + file)
print(data[17])

{'eng': {'lemma': '3D (three dimensional)', 'inflection': []}, 'swe': {'lemma': 'tre dimensionell', 'inflection': []}, 'pos': 'N?', 'status': '0', 'src': 'ICT', 'row': '7', 'comment': 'from data', 'synonyms': [], 'definition': None}


#### Basic data statistics

In [23]:
# Extract the English and Swedish lemmas 
simplified_data = [(entry["eng"]["lemma"], entry["swe"]["lemma"]) for entry in data]

# Convert data to pandas dataframe for easier manipulations
df_simplified_data = pd.DataFrame(simplified_data, columns=["english", "swedish"])
print("Number of terms in data", len(df_simplified_data))

# Group by english and swedish lemma to find number of duplicate entries
df_no_duplicates = df_simplified_data.groupby(df_simplified_data.columns.to_list(), as_index=False).size().sort_values(by="size", ascending=False)

# Number of unique entries
print("Number of unique entries", len(df_no_duplicates))
df_no_duplicates.head()

Number of terms in data 6883
Number of unique entries 6547


,english,swedish,size
839,Miscellaneous,Blandat,39
514,General,Allmänt,38
780,MISCELLANEOUS,BLANDAT,11
501,GENERAL,ALLMÄNT,11
1386,Test generation [**],Testgenerering [**],7


In [24]:
df_simplified_data.groupby(by=["swedish"], as_index=False).size().sort_values(by="size", ascending=False).head()

,swedish,size
0,,565
388,Blandat,39
307,Allmänt,38
357,BLANDAT,11
287,ALLMÄNT,11


565 terms without translation what to do with these??

In [25]:
df_simplified_data.groupby(by=["english"], as_index=False).size().sort_values(by="size", ascending=False).head()

,english,size
827,Miscellaneous,39
505,General,38
492,GENERAL,11
768,MISCELLANEOUS,11
72,Applications,7
